### Imports

In [52]:
import torch
import rasterio
import numpy as np
import cv2

### Spatial augmentation

In [ ]:
# TODO: give limits to random
def spatialAug1(sentinel1):
    vv = np.array(sentinel1.read(1))
    vh = np.array(sentinel1.read(2))

    i1 = np.random.randint(low=0, high=vv.shape[0]-2, size=None)
    i2 = np.random.randint(low=i1, high=vv.shape[0]-1, size=None)
    j1 = np.random.randint(low=0, high=vv.shape[1]-2, size=None)
    j2 = np.random.randint(low=j1, high=vv.shape[1]-1, size=None)

    vv_croped = vv[i1:i2, j1:j2]
    vh_croped = vh[i1:i2, j1:j2]

    vv_resized = cv2.resize(vv_croped, dsize=(512, 512))
    vh_resized = cv2.resize(vh_croped, dsize=(512, 512))
    fill = np.zeros((512,512))

    return np.stack((vv_resized, vh_resized, fill), axis=2)

def spatialAug2(sentinel2):
    r = np.array(sentinel2.read(3))
    g = np.array(sentinel2.read(2))
    b = np.array(sentinel2.read(1))

    i1 = np.random.randint(low=0, high=r.shape[0]-2, size=None)
    i2 = np.random.randint(low=i1, high=r.shape[0]-1, size=None)
    j1 = np.random.randint(low=0, high=r.shape[1]-2, size=None)
    j2 = np.random.randint(low=j1, high=r.shape[1]-1, size=None)

    r_croped = r[i1:i2, j1:j2]
    g_croped = g[i1:i2, j1:j2]
    b_croped = b[i1:i2, j1:j2]

    r_resized = cv2.resize(r_croped, dsize=(512, 512))
    g_resized = cv2.resize(g_croped, dsize=(512, 512))
    b_resized = cv2.resize(b_croped, dsize=(512, 512))

    return np.stack((r_resized, g_resized, b_resized), axis=2)
    

# Open the GeoTIFF image
sentinel1_image_path = "sentinel1_example.tif"
sentinel2_image_path = "sentinel2_example.tif"

sentinel1 = rasterio.open(sentinel1_image_path)
sentinel2 = rasterio.open(sentinel2_image_path)

spatialAug1(sentinel1)
spatialAug2(sentinel2)

### Sentinel 1 preprocessing

### Sentinel 2 preprocessing

### Models

In [53]:
resnet1  = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
resnet2  = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /home/charlie/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/charlie/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 88.1MB/s]
Using cache found in /home/charlie/.cache/torch/hub/pytorch_vision_v0.10.0


### Model training

In [ ]:
def criterion(x, y):
    sim = torch.nn.CosineSimilarity()
    l = np.empty(x.shape)
    for i in range(len(l)):
        lxyi = -torch.log(torch.exp(sim(x[i], y[i]))/(torch.sum(torch.exp(________)) + torch.sum(torch.exp(________))))
        lyxi = -torch.log(torch.exp(sim(y[i], x[i]))/(torch.sum(torch.exp(________)) + torch.sum(torch.exp(________))))
        l[i] = lxyi + lyxi
    return l

### Model evaluation